In [ ]:
%matplotlib widget
import rospy
import time
import actionlib
import rt2_assignment1.msg
from rt2_assignment1.srv import Command
from rt2_assignment1.srv import Velocity

import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from geometry_msgs.msg import Twist, Point
import ipywidgets as widgets
from rt2_assignment1.msg import PositionActionResult

auto_mode=False

reached=0
cancelled=0

task_time=0
update_time=[]

rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
rate = rospy.Rate(20)

pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)



In [ ]:
## START AND STOP RANDOM BEHAVIOUR

button1 = widgets.Button(description="Start",layout=widgets.Layout(widht='auto',grid_area='b1'),
                style=widgets.ButtonStyle(button_color='lightblue'))
                
button2 = widgets.Button(description='Stop',layout=widgets.Layout(width='auto', grid_area='b2'),
               style=widgets.ButtonStyle(button_color='salmon'))


def onStartCLicked(b): 
     global auto_mode, task_time
     ui_client("start")   
     auto_mode=True   
     task_time=rospy.Time.to_sec()
     print("\nStart moving")

def onStopCLicked(b): 
     global auto_mode, cancelled, task_time
     ui_client("stop")
     if auto_mode==True:  
          auto_mode=False    
          cancelled=cancelled+1
          #reset time
          task_time=0

     msg = Twist()
     msg.linear.x =0
     msg.angular.z=0
     pub.publish(msg)
          
     print("\nStop the robot")

widgets.GridBox(children=[button1, button2],
layout=widgets.Layout(
width='40%',
grid_template_rows='auto',
grid_template_columns='33%',
grid_template_areas='''
". b1 . b2 ."

''')
)




In [ ]:
button1.on_click(onStartCLicked)
button2.on_click(onStopCLicked)


In [ ]:
#SLIDERS

coeff_linear=1
coeff_angular=1

velocity_publisher = rospy.ServiceProxy('/set_velocity', Velocity)

linear_slider = widgets.FloatSlider(description='Linear velocity', min=-4.0, max=4.0)
display(linear_slider)

slider_angle = widgets.FloatSlider(description='Angular velocity', min=-4.0, max=4.0)
display(slider_angle)


def on_value_change(change):
    global coeff_linear, auto_mode, coeff_angular
    if auto_mode == False:
        msg=Twist()
        msg.linear.x = change['new']
        pub.publish(msg)
    else:
        coeff_linear=change['new']
        velocity_publisher(coeff_linear, coeff_angular)
    
def on_valueang_change(change):
    global coeff_angular, auto_mode, coeff_linear
    if auto_mode== False:
        msg=Twist()
        msg.angular.z = change['new']
        pub.publish(msg)
    else:
        coeff_angular=change['new']
        velocity_publisher(coeff_linear, coeff_angular)


linear_slider.observe(on_value_change, names='value')
slider_angle.observe(on_valueang_change, names='value')

In [ ]:
# BUTTONS TO DIRECT CONTROL THE ROBOT

forward = widgets.Button(description="Foreward",layout=widgets.Layout(widht='auto',grid_area='b1'),
                style=widgets.ButtonStyle(button_color='gray'))
                
backward = widgets.Button(description='Backward',layout=widgets.Layout(width='90%', grid_area='b2'),
               style=widgets.ButtonStyle(button_color='gray'))
turn_left = widgets.Button(description='Turn Left',layout=widgets.Layout(width='auto%', grid_area='b3'),
               style=widgets.ButtonStyle(button_color='gray'))
turn_right = widgets.Button(description='Turn Right',layout=widgets.Layout(width='auto%', grid_area='b4'),
               style=widgets.ButtonStyle(button_color='gray'))
stop = widgets.Button(description='Stop',layout=widgets.Layout(width='95%', grid_area='b5'),
               style=widgets.ButtonStyle(button_color='gray'))

def clk_foreward(b):
    global auto_mode, cancelled

    if auto_mode==True:
        ui_client("stop") 
        cancelled=cancelled+1
        auto_mode=False
    msg=Twist()
    msg.linear.x =0.5
    msg.angular.z=0
    pub.publish(msg)

def clk_backward(b):
    global auto_mode, cancelled

    if auto_mode==True:
        ui_client("stop") 
        cancelled=cancelled+1
        auto_mode=False
    msg=Twist()
    msg.linear.x =-0.5
    msg.angular.z=0
    pub.publish(msg)

def clk_turn_left(b):
    global auto_mode, cancelled

    if auto_mode==True:
        ui_client("stop") 
        cancelled=cancelled+1
        auto_mode=False
    msg=Twist()
    msg.angular.z =-0.5
    msg.linear.x =0
    pub.publish(msg)

def clk_turn_right(b):
    global auto_mode, cancelled

    if auto_mode==True:
        ui_client("stop") 
        cancelled=cancelled+1
        auto_mode=False
    ui_client("stop") 
    msg=Twist()
    msg.angular.z =0.5
    msg.linear.x =0
    pub.publish(msg)

def clk_stop(b):
    global auto_mode, cancelled
    
    if auto_mode==True:
        ui_client("stop") 
        cancelled=cancelled+1
        auto_mode=False
    msg=Twist()
    msg.linear.x =0
    msg.angular.z=0
    pub.publish(msg)

widgets.GridBox(children=[forward, backward, turn_left, turn_right, stop],
layout=widgets.Layout(
width='40%',
grid_template_rows='auto auto auto',
grid_template_areas='''
" .b1. "
"b3 . b4 "
" .b2. "
''')
)

In [ ]:
forward.on_click(clk_foreward)
backward.on_click(clk_backward)
turn_left.on_click(clk_turn_left)
turn_right.on_click(clk_turn_right)
stop.on_click(clk_stop)

In [ ]:
def clb_targets(msg):
  global reached, auto_mode, task_time, update_time
  if auto_mode==True:
    reached = reached+1
    update_time.append(rospy.Time.to_sec()-task_time)
  

jr.subscribe('/go_to_point/result',PositionActionResult, clb_targets)

In [ ]:

fig_time,ax_time= plt.subplots()    

# Update the plot    
def anim_time(i):
    global update_time, ax_time
    ax_time.hist(update_time,10, color='b',lw=10)
    ax_time.set_title("Time to reach a goal")
    
animation_time=animation.FuncAnimation(fig_time, anim_time, interval=100)

In [ ]:
# ANIMATION FOR REACHED TARGETS VS CANCELLED TARGETS

bar_fig1=plt.figure()
bar_ax1=bar_fig1.add_subplot(111)
targets=[]
langs = ['Reached targets', 'Cacelled targets']

def animate_bar(i):
    global reached, cancelled, targets
    targets=[reached, cancelled]
    bar_ax1.bar(langs, targets, color='b')
    bar_ax1.set_title("Reached targets vs Cancelled targets")
    # To be sure that the y limit is always updated conforming to the number of reached and cancelled targets
    if reached>=cancelled:
        bar_ax1.set_ylim(0, reached+10)
    else:
        bar_ax1.set_ylim(0, cancelled+10)
   # plt.bar(langs, targets)
    

ani_bar = animation.FuncAnimation(bar_fig1, animate_bar, interval=100)


In [ ]:
fig_velocity,ax_velocity=plt.subplots()
ax_velocity.set_xlim(0, 15)
ax_velocity.set_ylim(-1, 6)

linear_actual =ax_velocity.plot([],[],lw=2, color='red',label='actual linear velocity')
angular_actual =ax_velocity.plot([],[],lw=2, color='orange',label='actual angular velocity')
linear_cmd =ax_velocity.plot([],[], lw=2, color='blue',label='linear velocity set')
angular_cmd =ax_velocity.plot([],[], lw=2, color='purple',label='angular velocity set')

ax_velocity.legend()
ax_velocity.set_title("Actual Velocity vs set Velocity")
ax_velocity.set(xlabel='Time', ylabel='Velocity')
ax_velocity.set_xlabel('time')


# Cmd_vel values
cmd_linear=[]
cmd_angular=[]

# odometry values
odom_linear=[]
odom_angular=[]

time_odometry=[]
time_cmd_vel=[]


def twist_callback(vel):
    global cmd_angular, cmd_linear, time_cmd_vel

    cmd_linear.append(vel.linear.x)
    cmd_angular.append(vel.angular.z)

    if(len(time_cmd_vel) == 0):
        time_cmd_vel.append(0.05)
    else:
        time_cmd_vel.append(time_cmd_vel[len(time_cmd_vel)-1]+0.05)



def odometry_callback(msg):
    global odom_linear, odom_angular, time_odometry

    odom_linear.append(msg.twist.twist.linear.x)
    odom_angular.append(msg.twist.twist.angular.z)

    if(len(time_odometry) == 0):
        time_odometry.append(0.05)
    else:
        time_odometry.append(time_odometry[len(time_odometry)-1]+0.05)




def init_graph():
    global ax_velocity, linear_actual, angular_actual, linear_cmd, angular_cmd
    linear_actual.set_data([],[])
    angular_actual.set_data([],[])
    linear_cmd.set_data([],[])
    angular_cmd.set_data([],[])

    return [linear_actual, angular_actual, linear_cmd, angular_cmd]



def animate_velocity(i):
    global time_odometry, time_cmd_vel, odom_angular, odom_linear, linear_cmd, angular_cmd

    # Update dimension of the plot
    if len(time_odometry) and time_odometry[len(time_odometry)-1] > ax_velocity.get_xlim():
        ax_velocity.set_xlim(0,ax_velocity.get_xlim()+10)
        ax_velocity.figure.canvas.draw()

    linear_actual.set_data(time_odometry, odom_linear)
    angular_actual.set_data(time_odometry, odom_angular)

    linear_cmd.set_data(time_cmd_vel, linear_cmd)
    angular_cmd.set_data(time_cmd_vel, angular_cmd)

    return [linear_actual, angular_actual, linear_cmd, angular_cmd]


velocity_subscriber=jr.subscribe('/cmd_vel', Twist, twist_callback)
odom_subscriber=rospy.Subscriber('/odom', Odometry, odometry_callback)
anim_velocity= animation.FuncAnimation(fig_velocity, animate_velocity, init_func=init_graph, frames=100, interval=20, blit=True)


In [ ]:
# PLOT ODOMETRY OF THE ROBOT

fig_odom, ax_odom = plt.subplots()
ax_odom.set_xlim(( -6, 6))
ax_odom.set_ylim((-6, 6))
line_odom, = ax_odom.plot([], [], 'ro')

x_data=[]
y_data=[]

#Orientation
#x_direct=0
#y_direct=0

#x_prec=0
#y_prec=0

def init_odom():
    line_odom.set_data([], [])
    return (line_odom,)

def odom_callback(msg):
    global x_direct, y_direct

    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)

   # y_direct=(msg.pose.pose.orientation.y)
   # x_direct=(msg.pose.pose.orientation.x)

def animate_odom(i):
    global x_prec, y_prec, x_direct, y_direct

    line_odom.set_data(x_data, y_data)
    ax_odom.set_title("Robot position (x, y)")

    return (line_odom, ax_odom)

   # if (x_direct==x_prec and y_direct==y_prec):
   #     return (line,)

   # else:
   #     x_prec=x_direct
   #     y_prec=y_direct
   #     return (line, ax)
    

jr.subscribe('/odom', Odometry, odom_callback)

In [ ]:
anim_odom = animation.FuncAnimation(fig_odom,
animate_odom, init_func=init_odom,
frames=100, interval=20,
blit=True)
